In [3]:
#importing libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
import mysql.connector

In [4]:
# Step 1: Database connection setup
def connect_to_db():
    # For MySQL
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="chekith",
        database="twitter_data"
    )
    return conn


In [10]:
# Step 2: Read the CSV to get Twitter profile URLs
def load_profile_urls():
    df = pd.read_csv("C:\\Users\\user\\OneDrive\\Desktop\\New folder\\twitter_profiles_data.csv", header=None)  # Read without headers
    return df[0].tolist()  # Assuming URLs are in the first column

In [14]:
# Step 3: Scrape Twitter profile details
def scrape_twitter_profile():
    response = requests.get("https://drive.google.com/file/d/1PLYwrGn5YApyWU2QpjbdhM6tea0HuGq7/view")
    soup = BeautifulSoup(response.text, 'html.parser')
    profile_data = {
        "Bio": None,
        "Following Count": None,
        "Followers Count": None,
        "Location": None,
        "Website": None
    }
    # Extracting the bio
    bio_tag = soup.find('div', {'data-testid': 'UserDescription'})
    profile_data['Bio'] = bio_tag.text.strip() if bio_tag else None

    # Extracting the following count
    following_tag = soup.find('a', {'href': f"{url}/following"})
    profile_data['Following Count'] = following_tag.find('span').text.strip() if following_tag else None

    # Extracting the followers count
    followers_tag = soup.find('a', {'href': f"{url}/followers"})
    profile_data['Followers Count'] = followers_tag.find('span').text.strip() if followers_tag else None

    # Extracting location
    location_tag = soup.find('span', {'data-testid': 'UserLocation'})
    profile_data['Location'] = location_tag.text.strip() if location_tag else None

    # Extracting website
    website_tag = soup.find('a', {'data-testid': 'UserUrl'})
    profile_data['Website'] = website_tag['href'] if website_tag else None

    return profile_data

In [1]:
def insert_data_to_db(conn, data):
    cursor = conn.cursor()
    insert_query = """
    INSERT INTO profiles (url, bio, following_count, followers_count, location, website)
    VALUES (%s, %s, %s, %s, %s, %s);
    """
    cursor.execute(insert_query, (
        data['URL'], 
        data['Bio'], 
        data['Following Count'], 
        data['Followers Count'], 
        data['Location'], 
        data['Website']
    ))
    conn.commit()

    